In [3]:
"""
Convert the netCDF files to Geotiff and metadata file.


"""

SCRIPT_NAME = "Y2018M08D16_RH_Convertt_Geotiff_V01"
OUTPUT_VERSION = 1

EC2_INPUT_PATH = "/volumes/data/Y2018M08D08_RH_S3_EC2_V01/output_V01/"

In [4]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M08D16 UTC 10:23


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [59]:
import os
import netCDF4

In [50]:
def filename_to_dict(filename):
    values = filename.split("_")
    
    value_length = len(values)
    
    if value_length == 5:
        keys = ["floodtype","climate","model","year","returnperiod"]
        dictje = dict(zip(keys,values))
    elif value_length == 6:
        keys = ["floodtype","climate","subsidence","year","returnperiod","returnperiod_decimal"]
        dictje = dict(zip(keys,values))
    elif value_length == 8:
        keys = ["floodtype","climate","subsidence","year","returnperiod","returnperiod_decimal","model","unknown"]
        dictje = dict(zip(keys,values))
    else:
        print("error")
    print(dictje)          
    return dictje

In [54]:
ID = 0
master_dict = {}
for root, dirs, files in os.walk(EC2_INPUT_PATH):
    for one_file in files:
        if one_file.endswith("nc"):
            file_dict = {}
            file_dict["root"] = root
            file_dict["filename"] = one_file
            filename , extension = one_file.split(".")
            file_dict["properties"] = filename_to_dict(filename)
            master_dict[ID] = file_dict
            ID += 1

{'floodtype': 'inuncoast', 'returnperiod_decimal': '0', 'climate': 'rcp4p5', 'subsidence': 'wtsub', 'year': '2080', 'returnperiod': 'rp0050'}
{'floodtype': 'inuncoast', 'returnperiod_decimal': '0', 'climate': 'rcp4p5', 'subsidence': 'wtsub', 'year': '2050', 'returnperiod': 'rp0050'}
{'floodtype': 'inuncoast', 'returnperiod_decimal': '0', 'climate': 'rcp4p5', 'subsidence': 'wtsub', 'year': '2080', 'returnperiod': 'rp0005'}
{'floodtype': 'inuncoast', 'returnperiod_decimal': '5', 'climate': 'rcp4p5', 'subsidence': 'wtsub', 'year': '2050', 'returnperiod': 'rp0001'}
{'floodtype': 'inuncoast', 'returnperiod_decimal': '0', 'climate': 'rcp4p5', 'subsidence': 'wtsub', 'year': '2030', 'returnperiod': 'rp0050'}
{'floodtype': 'inuncoast', 'returnperiod_decimal': '0', 'climate': 'rcp4p5', 'subsidence': 'wtsub', 'year': '2030', 'returnperiod': 'rp0005'}
{'floodtype': 'inuncoast', 'returnperiod_decimal': '0', 'climate': 'rcp4p5', 'subsidence': 'wtsub', 'year': '2030', 'returnperiod': 'rp0250'}
{'floo

In [55]:
master_dict

{0: {'filename': 'inuncoast_rcp4p5_wtsub_2080_rp0050_0.nc',
  'properties': {'climate': 'rcp4p5',
   'floodtype': 'inuncoast',
   'returnperiod': 'rp0050',
   'returnperiod_decimal': '0',
   'subsidence': 'wtsub',
   'year': '2080'},
  'root': '/volumes/data/Y2018M08D08_RH_S3_EC2_V01/output_V01/coastal/inun_subsidence_v2_95/RCP_SUB/rcp4p5'},
 1: {'filename': 'inuncoast_rcp4p5_wtsub_2050_rp0050_0.nc',
  'properties': {'climate': 'rcp4p5',
   'floodtype': 'inuncoast',
   'returnperiod': 'rp0050',
   'returnperiod_decimal': '0',
   'subsidence': 'wtsub',
   'year': '2050'},
  'root': '/volumes/data/Y2018M08D08_RH_S3_EC2_V01/output_V01/coastal/inun_subsidence_v2_95/RCP_SUB/rcp4p5'},
 2: {'filename': 'inuncoast_rcp4p5_wtsub_2080_rp0005_0.nc',
  'properties': {'climate': 'rcp4p5',
   'floodtype': 'inuncoast',
   'returnperiod': 'rp0005',
   'returnperiod_decimal': '0',
   'subsidence': 'wtsub',
   'year': '2080'},
  'root': '/volumes/data/Y2018M08D08_RH_S3_EC2_V01/output_V01/coastal/inun_sub

In [56]:
for key, value in master_dict.items():
    print(key)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [68]:
def ncdump(nc_fid):
    '''ncdump outputs dimensions, variables and their attribute information.
    -------------------------------------------------------------------------------
    
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Args:
        nc_fid (netCDF4.Dataset) : A netCDF4 dateset object

    Returns:
        nc_attrs (list) : A Python list of the NetCDF file global attributes.
        nc_dims (list) : A Python list of the NetCDF file dimensions.
        nc_vars (list) : A Python list of the NetCDF file variables.
    '''

    nc_attrs = nc_fid.ncattrs()
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    return nc_attrs, nc_dims, nc_vars

In [101]:
def standardize_time(time_unit,times):
    """ Append standardize time to list
    -------------------------------------------------------------------------------
    
    The netCDF results of the university of Utrecht consist of multiple time 
    formats. 
    
    Args:
        time_unit (string) : units as provided by the netCDF4 file. 
        times (list) : list of time in units provided in time_units (e.g. days).
    
    Returns:
        standardized_time (list) : list of normalized times in datetime format.
    
    """
    
    standardized_time =[]
    for time in times:
        if time_unit == ("days since 1900-01-01 00:00:00") or (time_unit =="Days since 1900-01-01"):
            standardized_time.append(datetime.datetime(1900,1,1) + datetime.timedelta(days=time))
        elif time_unit == "days since 1901-01-01 00:00:00":
            standardized_time.append(datetime.datetime(1901,1,1) + datetime.timedelta(days=time))
        elif time_unit == "Days since 1960-01-01 00:00:00" or time_unit == "Days since 1901-01-01":
            standardized_time.append(datetime.datetime(1901,1,1) + datetime.timedelta(days=time))
        else:
            raise("Error, unknown format:",time_unit)
            standardized_time.append(-9999)
    return standardized_time

In [111]:
def convert_netcdf_geotiff(dictje):
    """ Convert netcdf to geotiff
    
    Args:
        dictionary with root, filename and properties.
    
    """
    
    input_path = os.path.join(dictje["root"],dictje["filename"])
    nc_fid = netCDF4.Dataset(input_path, 'r')
    nc_attrs, nc_dims, nc_vars = ncdump(nc_fid)
    parameter = nc_vars[-1]
    lats = nc_fid.variables['lat'][:]  # extract/copy the data
    lons = nc_fid.variables['lon'][:]
    times = nc_fid.variables['time'][:]
    time_unit = nc_fid.variables["time"].getncattr("units")
    print(time_unit)
    standardized_time = standardize_time(time_unit,times)
    
    i = 0
    Z = nc_fid.variables[parameter][i, :, :]
    
    return times, time_unit, Z
    

In [112]:
for ID, dictje in master_dict.items():
    if ID < 3:
        times, time_unit, Z = convert_netcdf_geotiff(dictje)

Days since 1960-01-01 00:00:00


MemoryError: 

In [87]:
time_unit

'Days since 1901-01-01'

In [108]:
times = [65378.5]
time_unit = "days since 1900-01-01 00:00:00"
standardize_time(time_unit,times)

[datetime.datetime(2078, 12, 31, 12, 0)]